<a href="https://colab.research.google.com/github/TobiAdeniji94/Adeniji_Tobi/blob/master/Penguins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Tuner using Comet.ML

This notebook aims to show a simple example of how to use the Keras Tuner to find the best set of hyperparameters to train a neural network model.

In addition, this notebook synchronizes the experiments with a Comet.ML project.

This notebook uses the [penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data) and installs the `keras-tuner` library.

<img src='https://imgur.com/orZWHly.png' alt='Penguins dataset'>

In [11]:
!pip install comet_ml keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing Libraries and Initializing Comet

Here we initialize Comet and create the Experiment we will use to log everything that happens. Here you'll need your Comet API_KEY.

In [21]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tempfile
import tensorflow as tf
import urllib.request

from comet_ml import Experiment
from keras_tuner import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers, losses, metrics, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import comet_ml
comet_ml.init()

experiment = Experiment(
    project_name="Penguins",
    auto_metric_logging=True,
    auto_param_logging=True,
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
)

COMET INFO: Comet API key is valid
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: tensorboard, keras, tensorflow. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tobiadeniji94/penguins/b8426d19f35042009ab18849e281be7b
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WAR

## Preparing the data

Let's start by downloading the raw dataset locally. Some of the examples in the TensorFlow documentation use this same dataset so I'm using the same file they use.

In [22]:
DATA_DIRECTORY = tempfile.mkdtemp(prefix="keras-tuner-data")
DATA_FILEPATH = os.path.join(DATA_DIRECTORY, "penguins.csv")

urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

df = pd.read_csv(DATA_FILEPATH)

experiment.log_table(filename="penguins.csv", tabular_data=df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


# Pipeline

We can now prepare a pipeline with all the transformations that we want to apply to the different fields of the dataset.

[Here](https://twitter.com/svpino/status/1429730545618112517?s=20) you can find more information about pipelines.

In [23]:
numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]

numerical_preprocessor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_preprocessor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", numerical_preprocessor, numerical_columns),
        ("categorical", categorical_preprocessor, ["island"]),
    ]
)

# Split and Transform

We can now split and transform the data.

Notice how we should split the data before preprocessing it. If we run the preprocessing pipeline before splitting the data, we will be [leaking the test data into the training process](https://twitter.com/svpino/status/1425019257449025536?s=20). 

In [24]:
y = df.species
X = df.drop(["species", "sex"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.20, 
    random_state=42
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

## Build the model

We need a function that builds our model. For this example, we are going to build a simple dense network.

There are three different hyperparameters that we want to tune:

* The number of units of the first dense layer. We want to try `4`, `8`, and `12`.
* The number of units of the second dense layer. We are also trying `4`, `8`, and `12`.
* The learning rate. Here we want to try `1e-2` and `1e-3`.

Notice how we are instrumenting our model with a couple of `hp.Int()` an `hp.Choice()` function. This is how the Tuner knows which parameters to tune.



In [25]:
def _model(hp):
    model = keras.Sequential([
        layers.Dense(
            hp.Int("dense_1_units", min_value=4, max_value=12, step=4, default=8),
            input_shape=(X_train.shape[1],)
            
        ),
        layers.Dense(
            hp.Int("dense_2_units", min_value=4, max_value=12, step=4, default=8), 
            activation="relu"
        ),
        layers.Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

# Hyperparameter tuning

We are now ready to start searching for the best hyperparameter values.

First, let's instantiate the tuner. Here we are using the [`RandomSearch`](https://keras.io/api/keras_tuner/tuners/random/) strategy. (At this time, the Keras Tuner also supports [`BayesianOptimization`](https://keras.io/api/keras_tuner/tuners/bayesian/), [`Hyperband`](https://keras.io/api/keras_tuner/tuners/hyperband/), and [`Sklearn`](https://keras.io/api/keras_tuner/tuners/sklearn/).)

The way the tunner will determine the best model is by looking at the validation accuracy and we are going to be running 10 trials.

Then, we can kick off the search. The signature of the `search()` function is the same as the `model.fit()` function in Keras.

Finally, we can print the results of the tuning process. They will be sorted with the best hyperparameter values at the top.

In [26]:
tuner = RandomSearch(
    _model,
    objective="val_accuracy",
    max_trials=10,
    overwrite=True,
    directory="keras-tuner",
    project_name="keras-tuner-example",
)

tuner.search_space_summary()

tuner.search(
    X_train[:,:], 
    to_categorical(y_train), 
    epochs=5, 
    validation_data=(X_test[:,:], to_categorical(y_test))
)

tuner.results_summary()

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.28985506296157837

Best val_accuracy So Far: 0.9855072498321533
Total elapsed time: 00h 00m 20s
Results summary
Results in keras-tuner/keras-tuner-example
Showing 10 best trials
Trial summary
Hyperparameters:
dense_1_units: 8
dense_2_units: 8
learning_rate: 0.01
Score: 0.9855072498321533
Trial summary
Hyperparameters:
dense_1_units: 12
dense_2_units: 12
learning_rate: 0.01
Score: 0.9855072498321533
Trial summary
Hyperparameters:
dense_1_units: 4
dense_2_units: 8
learning_rate: 0.01
Score: 0.9710144996643066
Trial summary
Hyperparameters:
dense_1_units: 12
dense_2_units: 8
learning_rate: 0.01
Score: 0.9710144996643066
Trial summary
Hyperparameters:
dense_1_units: 4
dense_2_units: 12
learning_rate: 0.01
Score: 0.8695651888847351
Trial summary
Hyperparameters:
dense_1_units: 4
dense_2_units: 8
learning_rate: 0.001
Score: 0.739130437374115
Trial summary
Hyperparameters:
dense_1_units: 12
dense_2_units: 8
learning_rate: 0.001
Score: 0.69565218

# Making predictions

We can use the best model that the tuner found to make predictions on the test set.

We then create a table with the test data, the target, and the predictions and log it to the Experiment.


In [27]:
best_model = tuner.get_best_models(num_models=1)[0]
y_pred = np.argmax(best_model.predict(X_test), axis=-1)
accuracy = np.sum(y_pred == y_test) / len(y_test) * 100
print(f"Accuracy: {accuracy:.2f}")

results = pd.DataFrame(np.concatenate((
    X_test, 
    np.expand_dims(y_test, axis=1), 
    np.expand_dims(y_pred, axis=1)), axis=1), columns=[
        "Culmen Length", "Culmen Depth", "Flipper Length", "Body Mass", 
        "Island - Biscoe", "Island - Dream", "Island - Torgersen", 
        "Species", "Prediction"]
)

experiment.log_table(filename="results.csv", tabular_data=results)
experiment.display("panels")
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tobiadeniji94/penguins/06df69dabcca45e2a5a02e59d6db1452
COMET INFO:   Parameters:
COMET INFO:     add_indicator       : False
COMET INFO:     categories          : auto
COMET INFO:     copy                : True
COMET INFO:     drop                : 1
COMET INFO:     dtype               : <class 'numpy.float64'>
COMET INFO:     fill_value          : 1
COMET INFO:     handle_unknown      : ignore
COMET INFO:     keep_empty_features : False
COMET INFO:     max_categories      : 1
COMET INFO:     min_frequency       : 1
COMET INFO:     missing_values      : nan
COMET INFO:     sparse              : deprecated
COMET INFO:     sparse